In [1]:
!nvidia-smi

Wed Oct 16 13:13:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:1B:00.0 Off |                  Off |
| 30%   24C    P8              29W / 300W |   1131MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# mols_without_atommaps = []
# for mol in mols:
#     copied_mol = copy.deepcopy(mol)
#     for atom in copied_mol.GetAtoms():
#         atom.SetAtomMapNum(0)
#     mols_without_atommaps.append(copied_mol)
# result = mol_split_fun(
#     np.array(mols_without_atommaps), sampler="scaffold", **astartes_kwargs
# )
# train, val, test = _unpack_astartes_result(result, include_val)

# Imports

In [2]:
# Standard library imports
import os
import math
from pathlib import Path

# Third-party imports
import pandas as pd
from lightning import pytorch as pl
import torch
from chemprop import data, featurizers, models, nn

# Local imports

# CUDA
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: True


# Chemprop Model

## Load Chemprop data & details

In [3]:
chemprop_dir = Path.cwd()
input_path = chemprop_dir / 'data' / 'regression.csv'
num_workers = 8

smiles_column = 'smiles'
target_columns = ['log_value']

df_input = pd.read_csv(input_path, index_col=0)
df_input.head()

,target,target_id,assay_id,smiles,type,relation,value,log_value,unit,molregno,...,journal,doi,description,assay_type,assay_test_type,assay_organism,assay_tissue,assay_cell_type,assay_confidence_score,data_validity_comment
6352,Cytochrome P450 3A4,CHEMBL340,CHEMBL3817398,NCCc1cccnc1,IC50,=,5000.0,3.698970,uM,704986,...,J Med Chem,10.1021/acs.jmedchem.5b01146,Inhibition of C-terminal four-histidine tagged...,A,NaN,Homo sapiens,NaN,NaN,9,Outside typical range
6353,Cytochrome P450 3A4,CHEMBL340,CHEMBL3817398,c1ccncc1,IC50,=,4000.0,3.602060,uM,13349,...,J Med Chem,10.1021/acs.jmedchem.5b01146,Inhibition of C-terminal four-histidine tagged...,A,NaN,Homo sapiens,NaN,NaN,9,Outside typical range
6354,Cytochrome P450 3A4,CHEMBL340,CHEMBL3817398,NCCCCCC(=O)NCc1cccnc1,IC50,=,1000.0,3.000000,uM,2088882,...,J Med Chem,10.1021/acs.jmedchem.5b01146,Inhibition of C-terminal four-histidine tagged...,A,NaN,Homo sapiens,NaN,NaN,9,Outside typical range
229,Cytochrome P450 3A4,CHEMBL340,CHEMBL664808,CC(C)C[C@H](C=O)NC(=O)[C@@H](NS(=O)(=O)c1ccc(F...,IC50,=,531.0,2.725095,uM,282942,...,J Med Chem,10.1021/jm0201924,Inhibition of Cytochrome P450 3A4 as BQ substrate,A,NaN,NaN,NaN,NaN,8,Outside typical range
8766,Cytochrome P450 3A4,CHEMBL340,CHEMBL4717445,O=C(Nc1cccc(Cl)c1)N1CCOc2ccc(-c3ccncc3)cc2C1,IC50,=,325.0,2.511883,uM,2524229,...,Bioorg Med Chem Lett,10.1016/j.bmcl.2016.04.018,Inhibition of human liver microsomes CYP3A4 us...,A,NaN,Homo sapiens,Liver,NaN,9,Outside typical range


In [5]:
print(len(df_input))
print(len(df_input.drop_duplicates()))

5184
5172


## Get molecule data and split

In [6]:
# Organize data for modeling
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

# Split data for train, val, and test
mols = [d.mol for d in all_data]
train_indices, val_indices, test_indices = data.make_split_indices(mols, 'scaffold_balanced', (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

## Retrieve MoleculeDataset and DataLoader

In [7]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Train
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Val
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

# Test
test_dset = data.MoleculeDataset(test_data, featurizer)

# DataLoaders
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

## Create MPNN

In [8]:
# Message passing and aggregation
mp = nn.BondMessagePassing()
agg = nn.MeanAggregation()

# Feed-forward network
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(output_transform=output_transform)

# Batch normalization
batch_norm = False

# Get metrics
metric_list = [nn.metrics.RMSEMetric(), nn.metrics.MAEMetric()] # Only the first metric is used for training and early stopping

# Create model
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): Identity()
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSELoss(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
)

## Train Model

In [9]:
# Load trainer for model
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator='auto',
    devices=[7],
    max_epochs=50,
)

# Train moel
trainer.fit(mpnn, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "../aten/src/ATen/cuda/CUDAContext.cpp":50, please report a bug to PyTorch. device=, num_gpus=

CUDA call was originally invoked at:

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/data/rbg/users/vincentf/wrickord/miniconda3/envs/ic50/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/data/rbg/users/vincentf/wrickord/miniconda3/envs/ic50/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/data/rbg/users/vincentf/wrickord/miniconda3/envs/ic50/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/data/rbg/users/vincentf/wrickord/miniconda3/envs/ic50/lib/python3.11/site-packages/tornado/platform/asyncio.py", l

## Get Model Results

In [ ]:
results = trainer.test(mpnn, test_loader)